[Home](../index.ipynb) / Web-Server: Minimal Webserver
***

# Minimal blocking Webserver

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 

#######################################
# Constants

# ID_LED = 13 # Esp8266: Croduino 
ID_LED = 25 # Esp32 HeltTec
# ID_LED = 14 # Esp32 NodeMCU, external LED

#======================================
SSID        = "ESP minimal http-server"
PASSWORD    = '123123123123' # default is 'micropythoN', attension: at least 8 chars, otherwise error: 'OSError: can't set AP config'
MAX_CLIENTS = 5 # Good maximal ammount of posiible clients


#######################################
# Init

from machine import Pin

led = Pin( ID_LED, Pin.OUT )


#======================================
# Typical code for setting up the controller as access point.
# At the end we get a server-socket listening for clients to connect:

import network
import socket


wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface

wlanAccessPoint.config( essid = SSID, password = PASSWORD ) # ESP-8266 and ESP-32
# wlanAccessPoint.config( essid = SSID, password = PASSWORD, max_clients = MAX_CLIENTS ) # ESP32
    
wlanAccessPoint.active( True )  # activate the interface

print( wlanAccessPoint.ifconfig()[0] )

# A internet stream server-socket: this server-socket creates the clientSockets.
serverSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

# To prevent "OSError: [Errno 98] Address already in use",
# when a previous execution has left the socket in a TIME_WAIT state,
# and so can’t be immediately reused:

#   SOL_SOCKET:   Socket Options That Apply to the Socket Layer
#   SO_REUSEADDR: Indicates if the local socket address can be reused.
#                 This option is supported by sockets with an address family
#                 of AF_INET or AF_INET6 and a type of SOCK_STREAM or SOCK_DGRAM.
#   1:            A value of 0 (default) disables the option.
#                 A non-zero value sets the option indicating the local socket address can be reused.

serverSocket.setsockopt( socket.SOL_SOCKET, socket.SO_REUSEADDR, 1 )

try:
    serverSocket.bind( ( '', 80 ) ) # everyone can connect
    serverSocket.listen( MAX_CLIENTS )
except Exception as _e:
    print( _e )


#######################################
# Extract request-path from header:
# For example:
#   In browser:      "http://192.168.4.1/led/On"
#   ==> strRequest:  "GET /led/On"
#   This function removes the "GET " and returns in this example "/led/On".

def getPathFromRequest( strRequest ) :
    iRequestPathStart = strRequest.find( "GET " ) # Every http request starts with "GET "

    if iRequestPathStart >= 0 :                    # followed by the request-path delimited with a ' ' charakter.
        iRequestPathStart += 4

        iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

        if iRequestPathEnd >= 0 :
            return strRequest[ iRequestPathStart : iRequestPathEnd ]

    return ""
    

#######################################
# Loop
# Listen for connecting clients and
# handle requests

strSetLed = "/led/On"
led.off()

try:
    while True:
        print( "Try ...", end=" " )
        try:
            # The return value is a pair (clientSocket, clientAddress)
            # where clientSocket is a new socket object where we can
            # receive data from and send data to:
            clientSocket, clientAddress = serverSocket.accept()

            strPath   = getPathFromRequest( str( clientSocket.recv( 1024 ) ) ) # read maximal 1024 bytes
            
            if   strPath == "/led/On" :
                strSetLed = "/led/Off"
                led.on()
                
            elif strPath == "/led/Off":
                strSetLed = "/led/On"
                led.off()

            print( "accepted: reques path [", strPath, "]" )
            
            # Send http-header:
            clientSocket.send   ( "HTTP/1.1 200 OK\n" )   # All ist OK: code 200
            clientSocket.send   ( "Content-Type: text/html; charset=UTF-8\n" ) # we send html and text is encoded in UTF-8
            clientSocket.send   ( "Connection: close\n\n" ) # We are (suspending and) closing the connection after sending the data.

            # Now send html content:
            clientSocket.send( '<!DOCTYPE html>' )
            clientSocket.send( '<html lang="de">' )
            clientSocket.send(     '<head>' )
            clientSocket.send(         '<meta charset="utf-8">' )
            clientSocket.send(         '<title>ESP minimal http-server</title>' )
            clientSocket.send(         '<meta name="viewport" content="width=device-width, initial-scale=1">' )
            #clientSocket.send(         '<link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">' )
            clientSocket.send(     '</head>' )
            clientSocket.send(     '<body>' )
            clientSocket.send(         '<h1><a href="/led/{0}">Set LED {0}</a></h1>'.format( "On" if (strSetLed == "/led/On") else "Off" ) )
            clientSocket.send(     '</body>' )
            clientSocket.send( '</html>' )

        except Exception as _e:
            print( _e )

        finally:
            try   : clientSocket.close()
            except: pass
        
        
except KeyboardInterrupt:
    pass # User has typed "Ctrl C"

finally:
    # Clean up:
    try   : serverSocket.close()
    except: pass

    try   : wlanAccessPoint.active( False )  
    except: pass
    import gc
    gc.collect()